In [2]:
import pandas as pd
import urllib.request
import csv
import zipfile
import subprocess
import numpy as np
import os
import io
from bs4 import BeautifulSoup
from pandas.tseries.offsets import DateOffset
import re


### Download raw files

You can use this section to get the monthly zip. It can be run fairly harmlessly as it shouldnt overwrite old versions you have

##### Function definition

In [404]:
def get_raw_marts(save_path='../raw/retail_sales_services/marts/staging/'
                  ,file_name='MARTS-mf.zip'):
    urllib.request.urlretrieve(
        'https://www.census.gov/econ/currentdata/datasets/MARTS-mf.zip'
        ,save_path+file_name
    )

In [565]:
def open_save_and_parse_mrts(mrts_path='../raw/retail_sales_services/marts/staging/'
                            ,file_name='MARTS-mf.zip'
                            ,save_path='../raw/retail_sales_services/marts/releases/'):
    zf = zipfile.ZipFile(mrts_path+file_name)
    output=str(subprocess.check_output('unzip -l '+mrts_path+file_name
                                ,shell=True),encoding='utf-8')
    
    file_context = []
    for row in output.split('\n')[3:-3]:
        file_context.append(
            pd.to_datetime(
                row.strip().split('  ')[1].strip()
            )
        )
    max_date = str(np.max(file_context))[:10].replace('-','_')
    
    save_path = save_path+'{}'.format(max_date)
    
    try:
        os.mkdir(save_path)
    except:
        ''
        
    sections = []
    with zf.open('README') as f:
        for row in f:
            if ' Section' in str(row):
                sections.append(str(row,'utf-8').replace(' Section','').strip())        
                
    path=save_path
    with zf.open('MARTS-mf.csv') as f:
        martsreader = csv.reader(io.TextIOWrapper(f,'utf8'), delimiter=','
                                ,quotechar='"',quoting=csv.QUOTE_MINIMAL)

        for line in martsreader:
            if len(line)==1:
                strip_value = line[0].strip()
            else:
                strip_value=''
            if strip_value in sections:
                name=strip_value.lower().replace(' ','_')
                try:
                    csvfile.close()
                except:
                    ''
                csvfile=open(path+name+'.csv','w')
                martswriter = csv.writer(csvfile,delimiter=',',quotechar='"'
                                        ,quoting=csv.QUOTE_MINIMAL)
            else:
                if len(line)>0:
                    martswriter.writerow(line)

    csvfile.close()                

##### Run Code

In [430]:
# with zf.open('README') as f:
#     readmereader = csv.reader(io.TextIOWrapper(f,'utf8'), delimiter='|')

In [431]:
# print(str(
#     io.BytesIO(
#         zf.open('README').read()
#     ).read()
# ,'utf-8'))

In [432]:
get_raw_marts()

In [433]:
open_save_and_parse_mrts()

In [47]:
save_path = '../raw/retail_sales_services/marts/releases/{}'.format(max_date)

## Get release history

In [440]:
def save_census_release_page(save_path='../raw/retail_sales_services/marts/releases/raw_release_history/meta/'
                  ,file_name='historical_marts.html'):
    urllib.request.urlretrieve(
        'https://www2.census.gov/retail/releases/historical/marts/'
        ,save_path+file_name
    )

In [561]:
def parse_census_release_page(save_path='../raw/retail_sales_services/marts/releases/raw_release_history/meta/'
                  ,file_name='historical_marts.html'):
    soup = BeautifulSoup(open(save_path+file_name), "html5lib")
    table_array=[]
    start_append=False
    for row in soup.find_all('table')[0].find_all('tr'):
        if start_append:
            current_row=[]
            for cell in row.find_all('td')[1:-1]:
                current_row.append(cell.text.strip())
            table_array.append(current_row)
        elif 'Parent Directory' in row.text:
            start_append=True
    all_files_df = pd.DataFrame(table_array[:-1],columns=['file_name','date_added','size'])
    
    all_files_df.loc[:,('file_type')]=all_files_df.loc[:,('file_name')].str.split('.').str[1]
    
    all_files_df.loc[:,('file_name_date')]=all_files_df.loc[:,('file_name')]\
    .apply(lambda x: re.sub('[^0-9]','', x))\
    .apply(lambda x: '20'+x[:2]+'-'+x[-2:] if (len(x)==4 and x[:2]<'40')
          else ( '19'+x[:2]+'-'+x[-2:] if len(x)==4 else ''))    
    
    return all_files_df

In [452]:
def retreve_save_release(file_name
                         ,save_path='../raw/retail_sales_services/marts/releases/raw_release_history/files/'):
    urllib.request.urlretrieve(
        'https://www2.census.gov/retail/releases/historical/marts/{}'.format(file_name)
        ,save_path+'{}'.format(file_name)
    )

In [1288]:
def parse_text_sic(text_file_path):
    
    table_dict={'1A':{0:[],1:[]},'1B':{0:[],1:[]}}
    start_append=False
    indices = [0,15,50,64,73]

    position=-1
    annual_fix=False
    current_table=''
    with open(text_file_path, encoding="latin-1") as f:
        for row in f:
            
            if '12 month total' in row:
                annual_fix=True
            if row.strip()[:5]=='TABLE':
                start_append=False
                if '1A' in row and not annual_fix:
                    current_table='1A'
                    position=-1
                    sales_as_of=pd.to_datetime(row.strip().split('--')[-1])
                elif '1B' in row and not annual_fix:
                    current_table='1B'
                    position=-1
                else:
                    break
                
            if len(row.strip())>0 and row.strip()[0] in ['2','1'] and len(row[0].strip())==0 and not annual_fix:
                position=position+1
                table_dict[current_table][position]=[]
                start_append=False
            


            if 'Retail trade, total' in row:                
                if annual_fix:
                    start_append=False
                    annual_fix=False
                else:
                    start_append=True
            
            if start_append:
#                print(row)
                table_dict[current_table][position].append(row)

            if row.strip()[:3]=='594':
                start_append=False
#    return table_dict
    split_position_dict={'1A':{},'1B':{}}
    for key in table_dict:
        for sub_key in table_dict[key]:
            split_position_dict[key][sub_key]=\
                max([x.rfind('...')+3 for x in table_dict[key][sub_key] if len(x)>0])
    
    for key in table_dict:
        for sub_key in table_dict[key]:
            fix_array=[]
            for row in table_dict[key][sub_key]:
                left=row[:split_position_dict[key][sub_key]]
                right=row[split_position_dict[key][sub_key]:]
                if len(left.strip())>0 or len(right.strip())>0:
                    fix_array.append([
                        left]+\
                        [x.strip() for x in right.split(' ') if len(x.strip())>0]
                    )
            table_dict[key][sub_key]=fix_array
            
    split_first_array_dict={'1A':{},'1B':{}}
    for key in table_dict:
        for sub_key in table_dict[key]:
            left=[x[0] for x in table_dict[key][sub_key]]
            split_first_array_dict[key][sub_key]=\
                max([len(re.search(re.compile("(?s:.*)[0-9]"), x).group()) for x in left
                    if re.search(re.compile("(?s:.*)[0-9]"), x)])


    for key in table_dict:
        for sub_key in table_dict[key]:
            fix_array=[]            
            for row in table_dict[key][sub_key]:
                left=row[0][:split_first_array_dict[key][sub_key]]
                right=row[0][split_first_array_dict[key][sub_key]:]
                temp =[left,right]+row[1:]
                temp = [x.strip() for x in temp]
                temp[1]=temp[1].replace('u"\u2026"','').rstrip('.')
                temp= temp[:2]+[x.replace(',','') for x in temp[2:]]
                
                fix_array.append(temp)
            
            table_dict[key][sub_key]=copy.deepcopy(fix_array)

    df_1A_0=pd.DataFrame(table_dict['1A'][0],columns=[
        'sic','description'
        ,sales_as_of
        ,sales_as_of-DateOffset(months=1)
        ,sales_as_of-DateOffset(months=2)]).set_index(['sic','description']).stack().reset_index()    
    df_1A_0.columns=['sic','description','as_of_date','value']
    df_1A_0.loc[:,('is_sa')]=1

    df_1A_1=pd.DataFrame(table_dict['1A'][1],columns=[
        'sic','description'
        ,sales_as_of-DateOffset(years=1)
        ,sales_as_of-DateOffset(months=1)-DateOffset(years=1)
        ]).set_index(['sic','description']).stack().reset_index()
    df_1A_1.columns=['sic','description','as_of_date','value']
    df_1A_1.loc[:,('is_sa')]=1

    df_1B_0=pd.DataFrame(table_dict['1B'][0],columns=[
        'sic','description'
        ,sales_as_of
        ,sales_as_of-DateOffset(months=1)
        ,sales_as_of-DateOffset(months=2)]).set_index(['sic','description']).stack().reset_index()
    df_1B_0.columns=['sic','description','as_of_date','value']
    df_1B_0.loc[:,('is_sa')]=0

    df_1B_1=pd.DataFrame(table_dict['1B'][1],columns=[
        'sic','description'
        ,sales_as_of-DateOffset(years=1)
        ,sales_as_of-DateOffset(months=1)-DateOffset(years=1)
    ]).set_index(['sic','description']).stack().reset_index()
    df_1B_1.columns=['sic','description','as_of_date','value']
    df_1B_1.loc[:,('is_sa')]=1        
    
    all_output=pd.concat([df_1A_0,df_1A_1,df_1B_0,df_1B_1])
    all_output=all_output.query('value!="(*)"')
#    all_output.loc[:,('value')]=all_output.loc[:,('value')].apply(lambda x: float(x) if x!='(*)' else float('NaN')).copy()
    all_output.loc[:,('release_as_of_date')]=sales_as_of
    return all_output

In [444]:
save_census_release_page()

In [506]:
file_names=parse_census_release_page()

In [545]:
all_text_files = file_names.query('file_type=="txt"').sort_values('file_name_date')#\
#['file_name'].tolist()

In [1108]:
# i=0
# for name in all_text_files:
#     print(name)
#     time.sleep(2)    
#     retreve_save_release(name)

In [1047]:
sic_second_format=\
    all_text_files.query('file_name_date>="1999-01" and file_name_date<="2001-04"')

In [1289]:
sic_second_format=\
    all_text_files.query('file_name_date>="1994-01" and file_name_date<="2001-04"')
i=0
for name in sic_second_format['file_name']:
#    print(name)
    save_path_string='../raw/retail_sales_services/marts/releases/raw_release_history/files/'
    current_df = parse_text_sic(save_path_string+name)
    
    if i==0:
        final_all_release=current_df.copy()
        i=i+1
    else:
        final_all_release=pd.concat([final_all_release,current_df.copy()])


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/tools/datetimes.py:116: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  unique_elements = set(islice(arg, check_count))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [1290]:
final_all_release.to_csv('../cleaned/retail_sales_services/marts/sic_history.csv',index=False)

In [1286]:
def parse_text_naics(text_file_path):
    
    table_dict={'1A':[],'1B':[]}
    start_append=False

#    annual_fix=False
    current_table=''
    with open(text_file_path, encoding="latin-1") as f:
        for row in f:
            

            if row.strip()[:5] in ['TABLE','TTABL'] or '1B.  ESTIMATED MONTHLY SALES FOR RETAIL AND FOOD SERVICES' in row:
                start_append=False
                if '1A' in row:
                    current_table='1A'
                    sales_as_of=pd.to_datetime(row.strip().split('--')[-1])
                elif '1B' in row:
                    current_table='1B'
                else:
                    break            


            if 'Retail & food services, total' in row:
                start_append=True
            
            if start_append:
                table_dict[current_table].append(row)

            if row.strip()[:3]=='722':
                start_append=False

    split_position_dict={'1A':{},'1B':{}}
    for key in table_dict:
        split_position_dict[key]=\
            max([x.rfind('...')+3 for x in table_dict[key] if len(x)>0])
    
    
    for key in table_dict:
        fix_array=[]
        for row in table_dict[key]:
            left=row[:split_position_dict[key]]
            right=row[split_position_dict[key]:]
            if len(left.strip())>0 or len(right.strip())>0:
                fix_array.append([
                        re.sub(r'\([0-9]\)', '', left)]+\
                        [x.strip() for x in right.split(' ') if len(x.strip())>0]
                    )
            table_dict[key]=fix_array

    split_first_array_dict={'1A':{},'1B':{}}
    for key in table_dict:
        left=[x[0] for x in table_dict[key]]
        split_first_array_dict[key]=\
                max([len(re.search(re.compile("(?s:.*)[0-9]"), x).group()) for x in left
                    if re.search(re.compile("(?s:.*)[0-9]"), x)])

    for key in table_dict:
        fix_array=[]            
        for row in table_dict[key]:
            left=row[0][:split_first_array_dict[key]]
            right=row[0][split_first_array_dict[key]:]
            temp =[left,right]+row[1:]
            temp = [x.strip() for x in temp]
            temp[1]=temp[1].replace('u"\u2026"','').rstrip('.')
            temp= temp[:2]+[x.replace(',','') for x in temp[2:]]
                
            fix_array.append(temp)
            
        table_dict[key]=fix_array
        
    df_1A=pd.DataFrame(table_dict['1A'],columns=[
        'naics','description'
        ,sales_as_of
        ,sales_as_of-DateOffset(months=1)
        ,sales_as_of-DateOffset(months=2)        
        ,sales_as_of-DateOffset(years=1)
        ,sales_as_of-DateOffset(years=1)-DateOffset(months=1)        
    ]).set_index(['naics','description']).stack().reset_index()    
    df_1A.columns=['naics','description','as_of_date','value']
    df_1A.loc[:,('is_sa')]=1

    df_1B=pd.DataFrame(table_dict['1B'],columns=[
        'naics','description'
        ,'year'
        ,'pct_change_from_prior'
        ,sales_as_of
        ,sales_as_of-DateOffset(months=1)
        ,sales_as_of-DateOffset(months=2)        
        ,sales_as_of-DateOffset(years=1)
        ,sales_as_of-DateOffset(years=1)-DateOffset(months=1)        
    ]).set_index(['naics','description']).drop(['year','pct_change_from_prior'],axis=1)\
    .stack().reset_index()    
    
    df_1B.columns=['naics','description','as_of_date','value']
    df_1B.loc[:,('is_sa')]=0        
    
    all_output=pd.concat([df_1A,df_1B])
    all_output=all_output.query('value!="(*)"')
#    all_output.loc[:,('value')]=all_output.loc[:,('value')].apply(lambda x: float(x) if x!='(*)' else float('NaN')).copy()
    all_output.loc[:,('release_as_of_date')]=sales_as_of
    return all_output

In [1287]:
naics_format=\
    all_text_files.query('file_name_date>"2001-04"')
i=0
for name in naics_format['file_name']:
    print(name)
    save_path_string='../raw/retail_sales_services/marts/releases/raw_release_history/files/'
    current_df = parse_text_naics(save_path_string+name)
    
    if i==0:
        final_all_release=current_df.copy()
        i=i+1
    else:
        final_all_release=pd.concat([final_all_release,current_df.copy()])


rs0105.txt
rs0106.txt
rs0107.txt
rs0108.txt
rs0109.txt
rs0110.txt


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/tools/datetimes.py:116: FutureWarning: In the future, 'NAT == x' and 'x == NAT' will always be False.
  unique_elements = set(islice(arg, check_count))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

rs0111.txt
rs0112.txt
rs0201.txt
rs0202.txt
rs0203.txt
rs0204.txt
rs0205.txt
rs0206.txt
rs0207.txt
rs0208.txt
rs0209.txt
rs0210.txt
rs0211.txt
rs0212.txt
rs0301.txt
rs0302.txt
rs0303.txt
rs0304.txt
rs0305.txt
rs0306.txt
rs0307.txt
rs0308.txt
rs0309.txt
rs0310.txt
rs0311.txt
rs0312.txt
rs0401.txt
rs0402.txt
rs0403.txt
rs0404.txt
rs0405.txt
rs0406.txt
rs0407.txt
rs0408.txt
rs0409.txt
rs0410.txt
rs0411.txt
rs0412.txt
rs0501.txt
rs0502.txt
rs0503.txt
rs0504.txt
rs0505.txt
rs0506.txt
rs0507.txt
rs0508.txt
rs0509.txt
rs0510.txt
rs0511.txt
rs0512.txt
rs0601.txt
rs0602.txt
rs0603.txt
rs0604.txt
rs0605.txt
rs0606.txt
rs0607.txt
rs0608.txt
rs0609.txt
rs0610.txt
rs0611.txt
rs0612.txt
rs0701.txt
rs0702.txt
rs0703.txt
rs0704.txt
rs0705.txt
rs0706.txt
rs0707.txt
rs0708.txt
rs0709.txt
rs0710.txt
rs0711.txt
rs0712.txt
rs0801.txt
rs0802.txt
rs0803.txt
rs0804.txt
rs0805.txt
rs0806.txt
rs0807.txt
rs0808.txt
rs0809.txt
rs0810.txt
rs0811.txt
rs0812.txt
rs0901.txt
rs0902.txt
rs0903.txt
rs0904.txt
rs0905.txt

In [1291]:
final_all_release.to_csv('../cleaned/retail_sales_services/marts/naics_history.csv',index=False)

In [1]:
'https://www.census.gov/retail/mrts/mrtshist.html'

'https://www.census.gov/retail/mrts/mrtshist.html'

In [15]:
def save_sic_release(file_name
                     ,save_file_name
                    ,save_path='../raw/retail_sales_services/marts/releases/raw_sic/'
    ):
    urllib.request.urlretrieve(
        'https://www2.census.gov/retail/releases/historical/mrts/'+file_name
        ,save_path+save_file_name
    )

In [16]:
from tqdm import tqdm

In [18]:
for i in tqdm(range(67,102)):
    year_value=str(i)[-2:]
    save_name=str(1900+i)
#    save_sic_release('sal{}.dat'.format(year_value),'sal{}.dat'.format(save_name))

100%|██████████| 35/35 [00:00<00:00, 43062.67it/s]


In [48]:
naics_sic_crosswalk=pd.read_excel('../raw/naics_sic/naics/2002_NAICS_to_1987_SIC.xls')

In [49]:
naics_sic_crosswalk.columns=['naics_2002','naics_2002_description'
                            ,'sic','sic_description']

In [53]:
naics_sic_crosswalk['naics_2002']=naics_sic_crosswalk['naics_2002'].astype(str)
naics_sic_crosswalk['sic']=naics_sic_crosswalk['sic'].astype(str).apply(lambda x: x.zfill(4))

In [57]:
naics_sic_crosswalk[naics_sic_crosswalk['naics_2002'].str.contains('^453')]

,naics_2002,naics_2002_description,sic,sic_description
1436,453110,Florists,5992,Florists
1437,453210,Office Supplies and Stationary Stores,5044,Office Equipment (office equipment sold via re...
1438,453210,Office Supplies and Stationery Stores,5049,"Professional Equipment and Supplies, NEC (reli..."
1439,453210,Office Supplies and Stationary Stores,5111,Printing and Writing Paper (printing and writi...
1440,453210,Office Supplies and Stationery Stores,5112,Stationery and Office Supplies (stationery and...
1441,453210,Office Supplies and Stationery Stores,5943,Stationery Stores
1442,453220,"Gift, Novelty, and Souvenir Stores",5199,"Nondurable Goods, NEC (curios, statuary, gifts..."
1443,453220,"Gift, Novelty, and Souvenir Stores",5947,"Gift, Novelty, and Souvenir Shops"
1444,453310,Used Merchandise Stores,5932,Used Merchandise Stores (except pawn shops)
1445,453910,Pet and Pet Supplies Stores,5149,"Groceries and Related Products, NEC (pet food ..."


In [28]:
naics_2012=pd.read_excel('../raw/naics_sic/naics/2-digit_2012_Codes.xls',skiprows=[1],
                        converters={'2012 NAICS US Code':str})
naics_2012.columns=['seq_num','naics_2012','naics_2012_description']

In [34]:
naics_2012['naics_2012']=naics_2012['naics_2012'].astype(str)

In [35]:
rsg=['4413','442','443','445','446','448','451','452','453','454','722']

In [55]:
naics_2012[naics_2012['naics_2012'].str.contains('^451')]

,seq_num,naics_2012,naics_2012_description
1201,1202,451,"Sporting Goods, Hobby, Musical Instrument, and..."
1202,1203,4511,"Sporting Goods, Hobby, and Musical Instrument ..."
1203,1204,45111,Sporting Goods Stores
1204,1205,451110,Sporting Goods Stores
1205,1206,45112,"Hobby, Toy, and Game Stores"
1206,1207,451120,"Hobby, Toy, and Game Stores"
1207,1208,45113,"Sewing, Needlework, and Piece Goods Stores"
1208,1209,451130,"Sewing, Needlework, and Piece Goods Stores"
1209,1210,45114,Musical Instrument and Supplies Stores
1210,1211,451140,Musical Instrument and Supplies Stores


In [36]:
naics_2012[naics_2012['naics_2012'].isin(rsg)]

,seq_num,naics_2012,naics_2012_description
1107,1108,4413,"Automotive Parts, Accessories, and Tire Stores"
1112,1113,442,Furniture and Home Furnishings Stores
1122,1123,443,Electronics and Appliance Stores
1142,1143,445,Food and Beverage Stores
1162,1163,446,Health and Personal Care Stores
1179,1180,448,Clothing and Clothing Accessories Stores
1201,1202,451,"Sporting Goods, Hobby, Musical Instrument, and..."
1215,1216,452,General Merchandise Stores
1225,1226,453,Miscellaneous Store Retailers
1247,1248,454,Nonstore Retailers


In [39]:
avaliable_sic_marts ={
 '52'          : 'Building mat, hardware, garden supply, and mobile home dealers'
,'521,3'         : 'Building mat and supply stores'
,'525'           : 'Hardware stores'
,'55 ex 554'  : 'Automotive dealers'
,'551,2,5,6,7,9'      : 'Motor vehicle and miscellaneous automotive dealers'
,'551'             : 'Motor vehicle (franchised)'
,'553'           : 'Auto and home supply stores'
,'57'          : 'Furniture, home furnishings, and'
,'571'           : 'Furniture and home furnishings'
,'5722,31,4'     : 'Household appliance, radio, TV,                                  '
,'5722'            : 'Household appliance stores'
,'53'          : 'General merchandise group stores'
,'531'           : 'Dept stores (ex leased depts)'
,'531'           : 'Dept stores (in leased depts)'
,'533'           : 'Variety stores'
,'539'           : 'Misc general mdse stores'
,'54'          : 'Food stores'
,'541'           : 'Grocery stores'
,'554'         : 'Gasoline service stations'
,'56'          : 'Apparel and accessory stores'
,'561'           : 'Mens and boys clothing'
,'562,3'         : 'Womens clothing, accessory'
,'565'           : 'Family clothing stores'
,'566'           : 'Shoe stores'
,'58'          : 'Eating and drinking places'
,'591'         : 'Drug and proprietary stores'
,'592'         : 'Liquor stores'
,'5961'       : 'Total mail order'}

maybe '5722':'443'?

In [ ]:
{  '553' : '4413'
,'571':'442'
,'5722,31,4':'443'
,'5722':'445'
 ,'592':'445'
 ,'591':'446'
,'56':'448'
,'nan':'451'
,'452':'571'
,'453':
,'454':'5961'
,'722':'58'
}

In [8]:
pd.read_csv('../raw/retail_sales_services/marts/releases/raw_sic/sal1967.dat',
        skiprows=5,delimiter='\\')

,Unnamed: 0,Unnamed: 1,Jan. 1967,Feb. 1967,Mar. 1967,Apr. 1967,May. 1967,Jun. 1967,Jul. 1967,Aug. 1967,Sep. 1967,Oct. 1967,Nov. 1967,Dec. 1967,TOTAL,Unnamed: 15
0,NaN,"RETAIL SALES, TOTAL",21341.0,20387.0,23987.0,23373.0,24881.0,25976.0,24504.0,24944.0,24953.0,24900.0,26199.0,31639.0,297084.0,NaN
1,NaN,TOTAL (EXCL. AUTOMOTIVE GROUP),17242.0,16485.0,19141.0,18561.0,19624.0,20485.0,19638.0,20416.0,20560.0,20144.0,21508.0,27186.0,240990.0,NaN
2,NaN,"DURABLE GOODS, TOTAL",6505.0,6221.0,7497.0,7517.0,8280.0,8667.0,7854.0,7718.0,7592.0,7952.0,7974.0,8521.0,92298.0,NaN
3,52,BUILDING MATERIALS GROUP STORES,818.0,785.0,973.0,1076.0,1233.0,1282.0,1204.0,1285.0,1224.0,1255.0,1144.0,1156.0,13435.0,NaN
4,"521,3,5","BUILDING MATERIALS,SUPPLY STORES, HARDWARE",743.0,699.0,849.0,908.0,1026.0,1101.0,1039.0,1118.0,1060.0,1089.0,995.0,984.0,11611.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"(3) The sales estimates for ""total mail order""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,(4) GAF represents stores which specialize in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,and miscellaneous shopping goods stores).,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,"(5) Data are adjusted for seasonal variations,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
current_year_string='1999'
with open('../raw/retail_sales_services/marts/releases/raw_sic/sal1999.dat') as f:
    sicreader = csv.reader(f, delimiter='\\'
                            ,quotechar='"',quoting=csv.QUOTE_MINIMAL)
    
    start_nsa=False
    start_sa=False
    
    this_years_dict = {'nsa':[],'sa':[]}
    
    for line in sicreader:
        if len(line)>1 and 'NOT ADJUSTED' in line[1]:
#            print('start non adjusted')
            start_nsa=True
        elif len(line)>1 and 'ADJUSTED' in line[1]:
#            print('start adjusted')
            start_sa=True
            start_nsa=False
        elif start_nsa:
            this_years_dict['nsa'].append(line)
        elif start_sa:
            if len(line[0])>0 and line[0][0]=='(':
                break
            else:
                this_years_dict['sa'].append(line)

#pd.DataFrame(this_years_dict['sa'])

In [49]:
import copy

In [115]:
def file_sic_transformer(current_year_string):

    with open('../raw/retail_sales_services/marts/releases/raw_sic/sal{0}.dat'.format(
    current_year_string)) as f:
        sicreader = csv.reader(f, delimiter='\\'
                                ,quotechar='"',quoting=csv.QUOTE_MINIMAL)

        start_nsa=False
        start_sa=False

        this_years_dict = {'nsa':[],'sa':[]}

        for line in sicreader:
            if len(line)>1 and 'NOT ADJUSTED' in line[1]:
    #            print('start non adjusted')
                start_nsa=True
            elif len(line)>1 and 'ADJUSTED' in line[1]:
    #            print('start adjusted')
                start_sa=True
                start_nsa=False
            elif start_nsa:
                if len(line[0])>0 and line[0][0]=='(':
                    break
                else:
                    this_years_dict['nsa'].append(line)
                
            elif start_sa:
                if len(line[0])>0 and line[0][0]=='(':
                    break
                else:
                    this_years_dict['sa'].append(line)

    #pd.DataFrame(this_years_dict['sa'])


    result_nsa = pd.DataFrame(this_years_dict['nsa'])
    new_header = result_nsa.iloc[0] #grab the first row for the header
    result_nsa = result_nsa[1:] #take the data less the header row
    result_nsa.columns = new_header #set the header row as the df header
    column_hold = copy.deepcopy(result_nsa.columns.tolist())
    column_hold[0]='sic_code'
    column_hold[1]='sic_description'
    column_hold=[x.replace('.','') for x in column_hold]
    column_hold=[pd.to_datetime(x) if current_year_string in x else x for x in column_hold]
    column_hold=[x+' '+ current_year_string if x=='TOTAL' else x for x in column_hold]
    result_nsa.columns=column_hold
    result_nsa.loc[:,('sic_code')]=result_nsa.loc[:,('sic_code')].astype(str)
    result_nsa.loc[:,('sic_code')]=result_nsa.apply(
        lambda x: x['sic_description'] if len(x['sic_code'])==0 else x['sic_code'], axis=1)
    result_nsa=result_nsa.drop(['','TOTAL'' '+ current_year_string],axis=1)

    result_nsa.loc[:,('is_adj')]=0

    result_nsa = result_nsa.set_index(['is_adj','sic_code','sic_description'])
    result_nsa = result_nsa.stack().reset_index()
    result_nsa.columns=['is_adj','sic_code','sic_description','as_of_date','value']

    if len(this_years_dict['sa'])>0:
        result_sa = pd.DataFrame(this_years_dict['sa'])
        if len(result_sa.columns)==16:
            result_sa.columns=column_hold
            result_sa=result_sa.drop('TOTAL'' '+ current_year_string,axis=1)
        else:
            result_sa.columns=column_hold[:-2]+['']
        result_sa.loc[:,('sic_code')]=result_sa.loc[:,('sic_code')].astype(str)
        result_sa.loc[:,('sic_code')]=result_sa.apply(
            lambda x: x['sic_description'] if len(x['sic_code'])==0 else x['sic_code'], axis=1)
        result_sa=result_sa.drop('',axis=1)
        result_sa.loc[:,('is_adj')]=1

        result_sa = result_sa.set_index(['is_adj','sic_code','sic_description'])
        result_sa = result_sa.stack().reset_index()
        result_sa.columns=['is_adj','sic_code','sic_description','as_of_date','value']
        return pd.concat([result_nsa,result_sa])
    else:
        return result_nsa

In [120]:
for i in range(1967,2001):
    year_string=str(i)
    output=file_sic_transformer(year_string)
    
    if i==1967:
        final_sic=output.copy()
    else:
        final_sic=pd.concat([final_sic,output.copy()])

In [122]:
final_sic.to_csv('../cleaned/retail_sales_services/marts/sic_monthly_history.csv')